## 1. 라이브러리

In [124]:

import olefile
import glob
import pandas as pd
import os
from datetime import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np


## 2. input 파트

In [125]:

# '*'는 임의 길이의 모든 문자열을 의미
flist = glob.glob('hwpdata/*.hwp')
# 검사 문서
target = 'target.hwp'
# score 척도를 정할 변수를 base로 설정
base = 0
# empty dataframe 생성
df = pd.DataFrame()
# 파일 리스트에서 파일이름만 추출하여 리스트로 생성
names = [os.path.basename(x) for x in flist]


In [ ]:
print(flist)

In [ ]:
print(names)

In [ ]:
print(df)

In [126]:
# 파일명을 title로 갖는 column생성
df['title'] = names
# 검사 문서의 df상 인덱스값을 index 변수에 저장
index = df[df['title'] == target].index[0]
# 빈 content 열 생성
content = []
# 코사인 유사도 항목 생성
df['cos_sim'] = np.nan
# 자카드 유사도 항목 생성
df['jac_sim'] = np.nan

In [132]:
print(df)

                                       title  cos_sim  jac_sim
0                         1. 민원상담위원 모집공고.hwp      NaN      NaN
1  1.전문임기제공무원(속기사, 마급) 경력경쟁채용시험 공고문(배포용).hwp      NaN      NaN
2                                   qweq.hwp      NaN      NaN
3                                 target.hwp      NaN      NaN


In [ ]:
print(index)

In [ ]:

for file in flist :
    # .hwp 파일을 읽기위해 olefile 임포트
    f = olefile.OleFileIO(file)
    # 읽어온 .hwp파일을 읽기 전용 파일 개체(BytesIO)로 encoded_text에 저장
    encoded_text = f.openstream('PrvText').read()
    # 읽기 전용 파일 개체를 'utf-16'방식으로 decoding
    decoded_text = encoded_text.decode('UTF-16')
    # decoding한 값을 content에 list 항목으로 저장
    content.append(decoded_text)

In [ ]:
# print(encoded_text)
# print(decoded_text)
print(content, sep='\n')
len(content)

In [ ]:
df['content'] = content

In [ ]:
df

In [ ]:
df['content'][0]

## 3. 벡터화 및 유사도 측정

In [ ]:
# 자연어를 벡터화하기 위한 Tf-idf 적용
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['content'])


In [ ]:
type(tfidf_matrix)

In [ ]:
# 문서의 각단어별로 문서에 대한 정보를 얼마나 갖고 있느냐를 뜻하는 값
print(tfidf_matrix) #  (a,b)는 array로 만들었을 때 각 뒤에 나오는 tf-idf 값의 위치
print(tfidf_matrix.shape)
print(tfidf.get_feature_names_out())
print(len(tfidf.get_feature_names_out()))

#### 3-1. 코사인 유사도

In [100]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [101]:
print(cosine_sim)

[[1.         0.05830587 0.05806165 0.05193522]
 [0.05830587 1.         0.01336071 0.12851837]
 [0.05806165 0.01336071 1.         0.03405808]
 [0.05193522 0.12851837 0.03405808 1.        ]]


In [102]:
sim_scores = list(enumerate(cosine_sim[index]))


In [103]:
print(sim_scores)

[(0, 0.05193521834770416), (1, 0.12851837004036096), (2, 0.03405808418379918), (3, 1.0)]


In [107]:
sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
for key, score in sim_scores :
    if score > base :
        print(df.iloc[key]['title'], score)

target.hwp 1.0
1.전문임기제공무원(속기사, 마급) 경력경쟁채용시험 공고문(배포용).hwp 0.12851837004036096
1. 민원상담위원 모집공고.hwp 0.05193521834770416
qweq.hwp 0.03405808418379918


-----------------------

#### 3-2. 자카드 유사도

In [70]:
df['content']

0    서울중앙지방법원 공고 제 40호\r\n\r\n서울중앙지방법원 민원상담위원 모집 공고...
1    서울중앙지방법원 공고 제2022 – 38호\r\n\r\n<전문임기제공무원(속기사, ...
2    \r\n故 윤관 제12대 대법원장 법원장(法院葬] 절차 안내   \r\n\r\n20...
3    서울중앙지방법원 공고 제2022 - 45호\r\n\r\n<전문임기제공무원(속기사, ...
Name: content, dtype: object

In [80]:
import konlpy
konlpy.__version__

'0.6.0'

In [81]:
from konlpy.tag import *

okt = Okt()


In [82]:
target = okt.morphs(df['content'][index])


In [115]:
for num_content in range(len(df)):
    compare = okt.morphs(df['content'][num_content])
    union = set(target).union(set(compare))
    # print('문서1과 문서2의 합집합 :',union)
    intersection = set(target).intersection(set(compare))
    # print('문서1과 문서2의 교집합 :',intersection)
    # print(f'{num_content}번 text와의 자카드 유사도 :',len(intersection)/len(union))
    score = len(intersection)/len(union)
    print(df.iloc[num_content]['title'], len(intersection)/len(union))


1. 민원상담위원 모집공고.hwp 0.15845070422535212
1.전문임기제공무원(속기사, 마급) 경력경쟁채용시험 공고문(배포용).hwp 0.27380952380952384
qweq.hwp 0.07120743034055728
target.hwp 1.0


In [119]:
df

,title,content
0,1. 민원상담위원 모집공고.hwp,서울중앙지방법원 공고 제 40호\r\n\r\n서울중앙지방법원 민원상담위원 모집 공고...
1,"1.전문임기제공무원(속기사, 마급) 경력경쟁채용시험 공고문(배포용).hwp","서울중앙지방법원 공고 제2022 – 38호\r\n\r\n<전문임기제공무원(속기사, ..."
2,qweq.hwp,\r\n故 윤관 제12대 대법원장 법원장(法院葬] 절차 안내 \r\n\r\n20...
3,target.hwp,"서울중앙지방법원 공고 제2022 - 45호\r\n\r\n<전문임기제공무원(속기사, ..."


------------